In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from utils import show_plot

In [ ]:
datos_personales = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{os.getenv('SALARIES_SPREADHSEET_ID')}/gviz/tq?tqx=out:csv&sheet=2020")

In [ ]:
datos_personales["sueldo_bruto_usd_oficial"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_bruto_usd_blue"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_blue"]
)
datos_personales["sueldo_neto_usd_oficial"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_neto_usd_blue"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_blue"]
)
datos_personales.tail()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_ars"],
    label="Sueldo Bruto en ARS",
    color="#FFAAAA",
    linestyle=":",
    marker="o",
)
ax1.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto en ARS",
    color="#FF0000",
    linestyle="-",
    marker="o",
)
ax1.set_ylabel("Sueldo en ARS")
ax1.set_xlabel("Fecha")
ax1.set_title("Evolución Histórica del Sueldo Bruto y Neto")

ax2 = ax1.twinx()

ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_usd_oficial"],
    label="Sueldo Bruto en USD (Oficial)",
    color="#AAAAFF",
    linestyle=":",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_usd_blue"],
    label="Sueldo Bruto en USD (Blue)",
    color="#AAFFAA",
    linestyle=":",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_usd_oficial"],
    label="Sueldo Neto en USD (Oficial)",
    color="#0000FF",
    linestyle="-",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_usd_blue"],
    label="Sueldo Neto en USD (Blue)",
    color="#00FF00",
    linestyle="-",
    marker="o",
)
ax2.set_ylabel("Sueldo en USD")

lines, labels, lines2, labels2 = (
    ax1.get_legend_handles_labels() + ax2.get_legend_handles_labels()
)
lines = lines + lines2
labels = labels + labels2
fig.legend(
    lines,
    labels,
    loc="upper left",
    bbox_to_anchor=(0.1, 1),
    bbox_transform=ax1.transAxes,
)

plt.setp(ax1.get_xticklabels(), rotation=60, ha="right")

plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
datos_personales["sueldo_ajustado"] = datos_personales["sueldo_neto_ars"].iloc[0]

for i in range(1, len(datos_personales)):
    if (
        datos_personales.loc[i, "puesto"] != datos_personales.loc[i - 1, "puesto"]
        or datos_personales.loc[i, "seniority"]
        != datos_personales.loc[i - 1, "seniority"]
    ):
        datos_personales.loc[i, "sueldo_ajustado"] = datos_personales.loc[
            i - 1, "sueldo_ajustado"
        ] + (
            datos_personales.loc[i, "sueldo_neto_ars"]
            - datos_personales.loc[i - 1, "sueldo_neto_ars"]
        )
    elif datos_personales.loc[i, "puesto"] != None:
        datos_personales.loc[i, "sueldo_ajustado"] = datos_personales.loc[
            i - 1, "sueldo_ajustado"
        ] * (1 + datos_personales.loc[i, "inflacion_del_mes_anterior"] / 100)
    else:
        datos_personales.loc[i, "sueldo_ajustado"] = 0.00
datos_personales["sueldo_ajustado"].tail()

In [ ]:
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_ajustado"],
    label="Sueldo Ajustado por Inflación",
    color="red",
    linestyle="--",
    marker="x",
)

show_plot(
    plot_id,
    "Mes",
    "Sueldo (ARS)",
    "Comparación de Sueldo Neto vs Sueldo Ajustado por Inflación",
)

In [ ]:
avg_salary_blue = (
    datos_personales.groupby(["puesto", "seniority"])["sueldo_bruto_usd_blue"]
    .mean()
    .reset_index()
)
avg_salary_oficial = (
    datos_personales.groupby(["puesto", "seniority"])["sueldo_bruto_usd_oficial"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(12, 8))

bar_width = 0.35
seniorities = avg_salary_blue["seniority"].unique()
indices = np.arange(len(seniorities))

for i, puesto in enumerate(avg_salary_blue["puesto"].unique()):
    group_blue = avg_salary_blue[avg_salary_blue["puesto"] == puesto]
    group_oficial = avg_salary_oficial[avg_salary_oficial["puesto"] == puesto]

    bar_positions = indices + i * bar_width

    plt.bar(
        bar_positions,
        group_blue["sueldo_bruto_usd_blue"],
        width=bar_width,
        label=f"{puesto} (Blue)",
        alpha=0.7,
        color="blue",
    )
    plt.bar(
        bar_positions + bar_width,
        group_oficial["sueldo_bruto_usd_oficial"],
        width=bar_width,
        label=f"{puesto} (Oficial)",
        alpha=0.7,
        color="orange",
    )

plt.title(
    "Comparación de Sueldo Bruto en USD (Blue vs Oficial) por Puesto y Seniority"
)
plt.xlabel("Seniority")
plt.ylabel("Sueldo Promedio en USD")
plt.xticks(indices + bar_width, seniorities, rotation=45)
plt.legend(title="Puesto y Tipo de Cambio")
plt.tight_layout()
plt.show()

In [ ]:
col_fecha = (
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)
)
col_fecha = pd.to_datetime(col_fecha)
col_mes_ordinal = col_fecha.map(lambda date: date.toordinal())

In [ ]:
X = np.array(col_mes_ordinal).reshape(-1, 1)
y_ars = datos_personales["sueldo_bruto_ars"]
y_usd_oficial = datos_personales["sueldo_bruto_usd_oficial"]
y_usd_blue = datos_personales["sueldo_bruto_usd_blue"]

In [ ]:
model_ars = LinearRegression()
model_usd_oficial = LinearRegression()
model_usd_blue = LinearRegression()

model_ars.fit(X, y_ars)
model_usd_oficial.fit(X, y_usd_oficial)
model_usd_blue.fit(X, y_usd_blue)
pass

In [ ]:
last_date = col_fecha.max()
future_dates = [last_date + pd.DateOffset(months=i) for i in range(1, 13)]
future_dates_ordinal = np.array([date.toordinal() for date in future_dates]).reshape(
    -1, 1
)
future_dates

In [ ]:
pred_ars = model_ars.predict(future_dates_ordinal)
print("Predicciones en ARS:\n", pred_ars, "\n")
pred_usd_oficial = model_usd_oficial.predict(future_dates_ordinal)
print("Predicciones en USD Oficial:\n", pred_usd_oficial, "\n")
pred_usd_blue = model_usd_blue.predict(future_dates_ordinal)
print("Predicciones en USD Blue:\n", pred_usd_blue)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

ax1.plot(
    future_dates,
    pred_ars,
    label="Sueldo Bruto en ARS",
    color="#FF0000",
    linestyle="-",
    marker="o",
)
ax1.set_title("Proyección del Sueldo Bruto en ARS")
ax1.set_ylabel("Sueldo en ARS")
ax1.set_xlabel("Fecha")
ax1.legend()

ax2.plot(
    future_dates,
    pred_usd_oficial,
    label="Sueldo Bruto en USD (Oficial)",
    color="#0000FF",
    linestyle="-",
    marker="o",
)
ax2.plot(
    future_dates,
    pred_usd_blue,
    label="Sueldo Bruto en USD (Blue)",
    color="#00FF00",
    linestyle="-",
    marker="o",
)
ax2.set_title("Proyección del Sueldo Bruto en USD")
ax2.set_ylabel("Sueldo en USD")
ax2.set_xlabel("Fecha")
ax2.legend()

plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()